In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 모델 구현 과정

###1. 리뷰 데이터 수집

In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

In [ ]:
!python /content/drive/Shareddrives/capstone/Elegant_Friends/src/crawling/playstore_crawling.py \
--input_url='https://play.google.com/store/apps/details?id=com.lguplus.mobile.cs&hl=ko&gl=KR&showAllReviews=true' \
--file_name=service_center \
--year=2020 \
--quarter=4 \
--only=False \

총 리뷰 수: 1293


### 2. 라벨링하기

- file_name은 크롤링 시 입력했던 file_name과 같아야 함 

In [ ]:
!python /content/drive/Shareddrives/capstone/Elegant_Friends/src/labeling/review_labeling.py \
--file_name=service_center \

###3. 감성 분류

- 3-1. train, dev, test로 파일 나누기

In [ ]:
!python /content/drive/Shareddrives/capstone/Elegant_Friends/src/ELBERT/make_data_set/train_dev_test.py \
--file_name=service_center \

train:  1034 , dev+test:  259
train:  129 , dev+test:  130


- 3-2. BERT 모델로 학습

In [ ]:
!pip uninstall tensorflow
!pip uninstall keras
!pip install tensorflow==1.14

In [ ]:
!python /content/drive/Shareddrives/capstone/Elegant_Friends/src/ELBERT/make_bert_model/run_classifier.py \
--task_name=asrc \
--do_train=true \
--do_eval=true \
--data_dir=/content/drive/Shareddrives/capstone/Elegant_Friends/rsc/ELBERT_data/train_data/service_center \
--vocab_file=/content/drive/Shareddrives/capstone/Elegant_Friends/rsc/ELBERT_data/conf/vocab.txt \
--bert_config_file=/content/drive/Shareddrives/capstone/Elegant_Friends/rsc/ELBERT_data/conf/bert_config.json \
--init_checkpoint=/content/drive/Shareddrives/capstone/Elegant_Friends/rsc/ELBERT_data/pretrained_model/wiki/model_output_512_model.ckpt-200000 \
--max_seq_length=128 \
--train_batch_size=32 \
--num_train_epochs=3.0 \
--learning_rate=3e-5 \
--do_lower_case=false \
--output_dir=/content/drive/Shareddrives/capstone/Elegant_Friends/rsc/ELBERT_data/fine_tuned_model/service_center \

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [ ]:
!python /content/drive/Shareddrives/capstone/Elegant_Friends/src/ELBERT/make_bert_model/run_classifier.py \
--task_name=asrc \
--do_predict=true \
--data_dir=/content/drive/Shareddrives/capstone/Elegant_Friends/rsc/crawling_data \
--vocab_file=/content/drive/Shareddrives/capstone/Elegant_Friends/rsc/ELBERT_data/conf/vocab.txt \
--bert_config_file=/content/drive/Shareddrives/capstone/Elegant_Friends/rsc/ELBERT_data/conf/bert_config.json \
--init_checkpoint=/content/drive/Shareddrives/capstone/Elegant_Friends/rsc/ELBERT_data/fine_tuned_model/model.ckpt-1079 \
--output_dir=/content/drive/Shareddrives/capstone/Elegant_Friends/rsc/ELBERT_data/sentiment_analyzed \

In [ ]:
!python /content/drive/Shareddrives/capstone/Elegant_Friends/src/ELBERT/make_bert_model/test_to_result.py \
--file_name=service_center \
--train=False \